In [1]:
from model import *
import sys

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# DEVICE = torch.device("cpu")

In [3]:
class VOCAB:
    pass

import json
with open('vocab.json') as f:
    itos_ = json.load(f)

stoi_ = {v:k for k,v in enumerate(itos_)}
VOCAB.itos = itos_
VOCAB.stoi = stoi_
n_vocab = len(VOCAB.itos)

In [4]:
import utils as ut

PAD_TOKEN = "#"
MASK_TOKEN = "[]"

class Batch:
    def __init__(self, src=None, trg=None, 
                 src_pad_mask=None, trg_pad_mask=None,
                 trg_mask=None, trg_y=None):
        self.src=src
        self.trg=trg
        self.src_pad_mask=src_pad_mask
        self.trg_pad_mask=trg_pad_mask
        self.trg_mask=trg_mask
        self.trg_y=trg_y
    def __repr__(self):
        return 'Batch{\n  src: %s\n  trg: %s}'%(itos(self.src.cpu().numpy()),
                                  itos(self.trg.cpu().numpy()))
    
def format_couplets_str(first, second='', stoi={}, mask_token="[]"):
    first = list(first.replace(' ', '').replace(',', '，').replace('.', '。'))
    second = [s.replace(' ', mask_token).replace('-', mask_token) for s in second]
    l = (len(first)-len(second))
    assert l >= 0
    second = second + [mask_token] * l
    return [stoi.get(s, s) for s in first], [stoi.get(s, s) for s in second]

def str_to_batch(first, second, stoi, mask_token):
    src, trg = format_couplets_str(first, second, stoi, mask_token)
    src = tc.LongTensor(src).unsqueeze(0)
    trg = tc.LongTensor(trg).unsqueeze(0)
    src_pad_mask = tc.ones_like(src, dtype=bool)
    trg_pad_mask = tc.ones_like(trg, dtype=bool)
    trg_mask = trg != stoi[mask_token]
    return Batch(src, trg, src_pad_mask,trg_pad_mask,trg_mask)

def itos(i, itos=VOCAB.itos):
    return ut.apply(''.join, ut.apply(itos.__getitem__, i), 
                    at=lambda c:not ut.iscollection(c[0][0]))

In [5]:
model = make_model(n_vocab, n_vocab).to(DEVICE)
model.load_state_dict(torch.load('model_state_share.pt'))

<All keys matched successfully>

In [6]:
from decode_search import *

def match_couplet_onepass(first, second=''):
    '一次性输出，模型调用1次，效果不好'
    batch = str_to_batch(first, second, VOCAB.stoi, MASK_TOKEN)
    model.eval()
    out = model(batch.src.to(DEVICE), batch.trg.to(DEVICE), 
                batch.src_pad_mask.unsqueeze(1).to(DEVICE), batch.trg_pad_mask.unsqueeze(1).to(DEVICE))
    _, ind = torch.max(model.generator(out), dim = -1)
    return itos(ind.cpu().numpy())

def match_couplet_greedy(first, second=''):
    batch = str_to_batch(first, second, VOCAB.stoi, MASK_TOKEN)
    model.eval()
    trg_pred = decode_greedy(model, batch.src.to(DEVICE), batch.trg.to(DEVICE)
                             , mask_int=VOCAB.stoi[MASK_TOKEN], pad_int=VOCAB.stoi[PAD_TOKEN])
    return itos(trg_pred.cpu().numpy())

def match_couplet_beam_1D(first, second='', beamsize=5):
    batch = str_to_batch(first, second, VOCAB.stoi, MASK_TOKEN)
    model.eval()
    results = beam_decode_engine(model, batch.src.to(DEVICE), batch.trg.to(DEVICE), beamsize, decode_1D_step
                             , mask_int=VOCAB.stoi[MASK_TOKEN], pad_int=VOCAB.stoi[PAD_TOKEN])
    return itos([r.cpu().numpy() for r in results])

def match_couplet_beam_margin(first, second='', beamsize=5):
    batch = str_to_batch(first, second, VOCAB.stoi, MASK_TOKEN)
    model.eval()
    results = beam_decode_engine(model, batch.src.to(DEVICE), batch.trg.to(DEVICE), beamsize, decode_margin_step
                             , mask_int=VOCAB.stoi[MASK_TOKEN], pad_int=VOCAB.stoi[PAD_TOKEN])
    return itos([r.cpu().numpy() for r in results])

def match_couplet_beam_2D(first, second='', beamsize=5):
    '条件概率版本'
    batch = str_to_batch(first, second, VOCAB.stoi, MASK_TOKEN)
    model.eval()
    results = beam_decode_engine(model, batch.src.to(DEVICE), batch.trg.to(DEVICE), beamsize, decode_2D_step
                             , mask_int=VOCAB.stoi[MASK_TOKEN], pad_int=VOCAB.stoi[PAD_TOKEN])
    return itos([r.cpu().numpy() for r in results])

def match_couplet_beam_2D_2(first, second='', beamsize=5):
    '联合分布概率版本'
    batch = str_to_batch(first, second, VOCAB.stoi, MASK_TOKEN)
    model.eval()
    results = decode_beam_2D_2(model, batch.src.to(DEVICE), batch.trg.to(DEVICE), beamsize
                             , mask_int=VOCAB.stoi[MASK_TOKEN], pad_int=VOCAB.stoi[PAD_TOKEN])
    return itos([r.cpu().numpy() for r in results])

def decode_margin_step2(model, src, trg, mask, padmask, beamsize):
    '''
    各候选位置选最可能的字 + 最可能的位置选候选字，伪二维搜索
    padmask为True的是有效字，mask为True的是句中的给定字（非mask）
    '''
    model.eval()
#     print(padmask.unsqueeze(0).unsqueeze(1))
    assert src.size(0) == 1
    out = model(src, trg, padmask.unsqueeze(0).unsqueeze(1), padmask.unsqueeze(0).unsqueeze(1))
    g = model.generator(out)
    s, ind = torch.max(g, dim = -1)
    ignore = mask | (~padmask)
    s[0][ignore] = -1e9
    snp = s[0].detach().cpu().numpy() 
    pos = max_n_1D(np.array(snp), min(len(snp), beamsize//2))
    result = []
    keep_pos = pos[:(~ignore).sum().item()]
    for p in keep_pos:
        r = tc.clone(trg)
        r[0, p] = ind[0][p].item()
        result.append(r)
    logps = snp[keep_pos]
    max_pos = pos[0]
    g = g[0, max_pos].detach().cpu().numpy()
    ind = max_n_1D(g, min(len(g), beamsize-len(result)))
    
    for i in ind:
        r = tc.clone(trg)
        r[0, max_pos] = i
        result.append(r)
#     print(itos([r.cpu().numpy() for r in result]))
    return result

def beam_decode_engine2(model, src, trg, beamsize, decodestep, mask_int, pad_int):
    '候选的概率由logp_of_trg确定（联合分布）'
    padmask = (src!=pad_int).squeeze()
    result = [trg]
    while True:
        candidates = {}
        for trg in result:
            mask = (trg!=mask_int).squeeze()
            if mask.all(): 
                return result
            cands = decodestep(model, src, trg, mask, padmask, beamsize)
            for c in cands:
                candidates[tuple(c[0].tolist())] = c #去重
        cands = list(candidates.values())
        logps = [logp_of_trg(model, src, trgi, (trgi!=mask_int).squeeze(0), padmask) for trgi in cands]
        maxinds = max_n_1D(np.array(logps), min(len(logps), beamsize))
        result = [cands[i] for i in maxinds]
#         print(itos([r.cpu().numpy() for r in result]))
    return result

def match_couplet_beam_margin2(first, second='', beamsize=5):
    batch = str_to_batch(first, second, VOCAB.stoi, MASK_TOKEN)
    model.eval()
    results = beam_decode_engine2(model, batch.src.to(DEVICE), batch.trg.to(DEVICE), beamsize, decode_margin_step2
                             , mask_int=VOCAB.stoi[MASK_TOKEN], pad_int=VOCAB.stoi[PAD_TOKEN])
    return itos([r.cpu().numpy() for r in results])

array=list
def print_match_all(first, second='', beamsize=5, file=sys.stdout):
    print("上联:", first, file=file)
    print("onepass(not good):", match_couplet_onepass(first, second), file=file)
    print("greedy :", match_couplet_greedy(first, second), file=file)
    print("beam_1D:\n", array(match_couplet_beam_1D(first, second, beamsize)), file=file)
    print("beam_2D:\n", array(match_couplet_beam_2D(first, second, beamsize)), file=file)
    print("beam_2D_2:\n", array(match_couplet_beam_2D_2(first, second, beamsize)), file=file)
    print("beam_margin:\n", array(match_couplet_beam_margin(first, second, beamsize)), file=file)
    print("beam_margin2:\n", array(match_couplet_beam_margin2(first, second, beamsize)), file=file)

In [7]:
firsts = ['天与云与山与水，上下一白',
          '两个黄鹂鸣翠柳',
          '无边落木萧萧下', 
          '月落乌啼霜满天', 
          '提刀上马，江山如画', 
          '兰亭临帖，行书如行云流水',
          '荒烟漫草的年头，就连分手都很沉默',
          '一条大河波浪宽，风吹稻花香两岸',
          '我欲成仙，快乐齐天',
          '只有刚强的人，才有神圣的意志，凡是战斗的人，才能取得胜利',
          '？',
         ]

In [ ]:
for f in firsts:
    print_match_all(f)

上联: 天与云与山与水，上下一白
onepass(not good): ['人和月和月和梅，左右同明']
greedy : ['人和地和地和天，古今同春']
beam_1D:
 [['人和梅和竹和梅，左右同清'], ['人和竹和竹和梅，左右同清']]
beam_2D:
 [['人和竹和竹和梅，左右同清'], ['人和竹和竹和梅，左右同青'], ['人和梅和竹和梅，左右同清'], ['人和梅和竹和梅，左右同青'], ['人和菊和竹和梅，左右同清']]
beam_2D_2:
 [['人和天和地和人，天地皆春'], ['人和天和地和人，大地皆春'], ['人和天和地和天，大地皆春'], ['人和天和地和天，天地皆春'], ['人和天和地和人，此地皆春']]
beam_margin:
 [['人和日和月和星，古今同辉'], ['人和日和月和风，左右同明'], ['人和日和月和风，古今同辉'], ['人和月和日和星，古今同辉'], ['人和月和日和月，古今同辉']]
beam_margin2:
 [['人和月和日和风，古今无穷'], ['人和事和地和天，多少无穷'], ['人和道和地和天，年少无穷'], ['人和天和地和人，年少无穷'], ['人和天和日和风，古今无穷']]
上联: 两个黄鹂鸣翠柳
onepass(not good): ['一双白鹭戏春花']
greedy : ['一双紫燕剪春风']
beam_1D:
 [['一行白鹭上青天']]
beam_2D:
 [['一行白鹭上青天'], ['一行白鹭上蓝天'], ['一行白鹭上青云'], ['一行白鹭上青霄'], ['一行白鹭上云天']]
beam_2D_2:
 [['一行白鹭上青天'], ['一行白鹭上苍天'], ['几行白鹭上青天'], ['几行白鹭上苍天'], ['一双白鹭上青天']]
beam_margin:
 [['一行白鹭上青天'], ['一双紫燕剪春风'], ['一行白鹭上青云'], ['一行白鹭上青霄'], ['一双白鹭上青天']]
beam_margin2:
 [['一行白鹭上青天'], ['一行白鹭上苍天'], ['一轮红日耀中天'], ['一双白鹭上青天'], ['一行白鹭上蓝天']]
上联: 无边落木萧萧下
onepass(not good): ['不尽飞花滚滚来']
greedy : ['不尽

In [11]:
print('输入上下联，或`q`退出')
print('如有下联用`|`隔开，下联空字用空格或减号占位')
print('输入例：\n白日依山尽\n白日依山尽|-河-海\n白日依山尽|明月\n')

while True:
    print("请输入：", end='')
    i = input().split('|')
    first = i[0]
    second = i[1] if len(i)>1 else ''
    if first.startswith('q'): break
    print_match_all(first, second)
    print('='*8)

输入上下联，或`q`退出
如有下联用`|`隔开，下联空字用空格或减号占位
输入例：
白日依山尽
白日依山尽|-河-海
白日依山尽|明月

请输入：

 床前明月光


上联: 床前明月光
onepass(not good): ['窗上美人香']
greedy : ['网上美人情']
beam_1D:
 [['梦里故人情'], ['梦里美人情'], ['笔下墨花香'], ['网上美人情'], ['梦里墨花香']]
beam_2D:
 [['梦里故人情'], ['梦里美人情'], ['梦里桂花香'], ['网上美人情'], ['梦里野花香']]
beam_2D_2:
 [['网上佳人佳'], ['网上丽人佳'], ['网上佳人句'], ['台上丽人佳'], ['网上佳人对']]
beam_margin:
 [['梦里美人情'], ['网上美人情'], ['梦里故人情'], ['陌上野花香'], ['梦里美人佳']]
beam_margin2:
 [['网上丽人佳'], ['网上佳人句'], ['台上丽人佳'], ['网上佳人对'], ['台上丽人和']]
请输入：

 q


In [ ]:
!python run.py